In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm

2023-09-06 23:27:03.078879: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 23:27:04.951936: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
class NN(pl.LightningModule):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1        = nn.Linear(input_size, 50)
        self.fc2        = nn.Linear(50, num_classes)
        self.loss_fn    = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

    # # Old way
    # # We don't need to perform zero_grad() and loss.backward() manually
    # def training_step(self, batch, batch_idx):
    #     x, y    = batch
    #     # Flatten image
    #     x       = x.reshape(x.shape[0], -1)
    #     logits  = self.forward(x)
    #     loss    = F.cross_entropy(logits, y)
    #     return loss

    # def validation_step(self, batch, batch_idx):
    #     x, y    = batch
    #     # Flatten image
    #     x       = x.reshape(x.shape[0], -1)
    #     logits  = self.forward(x)
    #     loss    = F.cross_entropy(logits, y)
    #     return loss

    # def test_step(self, batch, batch_idx):
    #     x, y    = batch
    #     # Flatten image
    #     x       = x.reshape(x.shape[0], -1)
    #     logits  = self.forward(x)
    #     loss    = F.cross_entropy(logits, y)
    #     return loss

    # We can use _common_step() to reduce code duplication
    def _common_step(self, batch, batch_idx):
        x, y    = batch
        # Flatten image
        x       = x.reshape(x.shape[0], -1)
        logits  = self.forward(x)
        loss    = F.cross_entropy(logits, y)
        return loss, logits, y

    def training_step(self, batch, batch_idx):
        loss, logits, y = self._common_step(batch, batch_idx)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, logits, y = self._common_step(batch, batch_idx)
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        loss, logits, y = self._common_step(batch, batch_idx)
        self.log('test_loss', loss)
        return loss


    def predict_step(self, batch, batch_idx):
        x, y    = batch
        # Flatten image
        x       = x.reshape(x.shape[0], -1)
        logits  = self.forward(x)
        preds   = torch.argmax(logits, dim=1)
        return preds

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-3)

In [4]:
entire_dataset = datasets.MNIST(
    root='dataset/',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)

In [5]:
train_ds, val_ds = random_split(entire_dataset, [55000, 5000])
test_ds = datasets.MNIST(
    root='dataset/',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

In [6]:
train_loader = DataLoader(dataset=train_ds,
                          batch_size=32,
                          num_workers=4,
                          shuffle=True)

val_loader = DataLoader(dataset=val_ds,
                        batch_size=32,
                        num_workers=4,
                        shuffle=False)

test_loader = DataLoader(dataset=test_ds,
                        batch_size=32,
                        num_workers=4,
                        shuffle=False)

In [7]:
model = NN(input_size=784, num_classes=10).to(device)

In [8]:
trainer = pl.Trainer(min_epochs=2, max_epochs=5, devices=[0], accelerator='gpu', precision=16)
trainer.fit(model, train_loader, val_loader)

/home/long/.local/lib/python3.8/site-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params
---------------------------------------------
0 | fc1     | Linear           | 39.2 K
1 | fc2     | Linear           | 510   
2 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
39.8 K    Trainable params
0         Non-trainable params
39.8 K    Total params
0.159     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [9]:
trainer.validate(model, val_loader)
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    0.10983605682849884    │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.10941261053085327    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.10941261053085327}]